In [ ]:
%matplotlib notebook


3. Magnetic source using duality
================================

Computing the $E$ and $H$ fields from a magnetic source using the
duality principle.

We know that we can get the magnetic fields from the electric fields using
Faraday's law, see `sphx_glr_gallery_magnetics_magnetic_field.py`.

However, what about computation the fields generated by a magnetic source?
There are two ways we can achieve that:

- **using the duality principle**, which is what we do in this example, or
- creating an electric loop source, see
  `sphx_glr_gallery_magnetics_magnetic_source_el_loop.py`.

``emg3d`` solves the following equation,

\begin{align}:label: maxwell

     \eta \mathbf{\hat{E}} - \nabla \times \zeta^{-1} \nabla \times
     \mathbf{\hat{E}} = -\mathbf{\hat{J}}^e_s ,\end{align}

where $\eta = \sigma - \mathrm{i}\omega\varepsilon$, $\zeta =
\mathrm{i}\omega\mu$, $\sigma$ is conductivity (S/m), $\omega=2\pi
f$ is the angular frequency (Hz), $\mu=\mu_0\mu_\mathrm{r}$ is magnetic
permeability (H/m), $\varepsilon=\varepsilon_0\varepsilon_\mathrm{r}$ is
electric permittivity (F/m), $\mathbf{\hat{E}}$ the electric field in the
frequency domain (V/m), and  $\mathbf{\hat{J}}^e_s$ source current.

This is the electric field due to an electric source. One can obtain the
magnetic field due to a magnetic field by substituting

- $\eta \leftrightarrow -\zeta$ ,
- $\mathbf{\hat{E}} \leftrightarrow -\mathbf{\hat{H}}$ ,
- $\mathbf{\hat{J}}^e_s \leftrightarrow \mathbf{\hat{J}}^m_s$ ,

which is called the **duality principle**.

Carrying out the substitution yields

\begin{align}:label: dualdip

    \zeta \mathbf{\hat{H}} - \nabla \times \eta^{-1} \nabla \times
    \mathbf{\hat{H}} = -\mathbf{\hat{J}}^m_s ,\end{align}

which is for a magnetic dipole. Changing it for a loop source adds a term
$\mathrm{i}\omega\mu$ to the source term, resulting in

\begin{align}:label: dualloop

    \zeta \mathbf{\hat{H}} - \nabla \times \eta^{-1} \nabla \times
    \mathbf{\hat{H}} = -\mathrm{i}\omega\mu\mathbf{\hat{J}}^m_s ;\end{align}

see `Dipoles and Loops
<https://empymod.readthedocs.io/en/stable/examples/educational/dipoles_and_loops.html>`_
for more information.

``emg3d`` is not ideal for the duality principle. Magnetic permeability is
implemented isotropically, and discontinuities in magnetic permeabilities can
lead to first-order errors in contrary to second-order errors for
discontinuities in conductivity. However, we can still abuse the code and use
it with the duality principle, at least for isotropic media.

The actual implemented equation in ``emg3d`` is a slightly modified version of
Equation :eq:`maxwell`, using the diffusive approximation
$\varepsilon=0$,

\begin{align}:label: dualdiff

    \mathrm{i}\omega \mu_0 \sigma \mathbf{\hat{E}} - \nabla \times
    \mu_r^{-1} \nabla \times \mathbf{\hat{E}} =
    -\mathrm{i}\omega\mu_0\mathbf{\hat{J}}_s .\end{align}

We therefore only need to interchange $\sigma$ with
$\mu_\mathrm{r}^{-1}$ or $\rho$ with $\mu_\mathrm{r}$ to get
from :eq:`dualdiff` to :eq:`dualloop`.

This is what we do in this example, for an arbitrarily rotated loop in a
homogeneous, isotropic fullspace. We compare the result to the semi-analytical
solution of ``empymod``. (The code ``empymod`` is an open-source code which can
model CSEM responses for a layered medium including VTI electrical anisotropy,
see `empymod.github.io <https://empymod.github.io>`_.)


In [ ]:
import emg3d
import empymod
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate as sint
from matplotlib.colors import SymLogNorm
plt.style.use('ggplot')

In [ ]:
def plot_data_rel(ax, name, data, x, vmin=-15., vmax=-7., mode="log"):
    """Plot function."""

    ax.set_title(name)
    ax.set_xlim(min(x)/1000, max(x)/1000)
    ax.set_ylim(min(x)/1000, max(x)/1000)
    ax.axis("equal")

    if isinstance(mode, str):
        if mode == "abs":
            cf = ax.pcolormesh(
                    x/1000, x/1000, np.log10(np.abs(data)), linewidth=0,
                    rasterized=True, cmap="viridis", vmin=vmin, vmax=vmax)
        else:
            cf = ax.pcolormesh(
                    x/1000, x/1000, data, linewidth=0, rasterized=True,
                    cmap="PuOr_r",
                    norm=SymLogNorm(linthresh=10**vmin,
                                    vmin=-10**vmax, vmax=10**vmax))
    else:
        cf = ax.pcolormesh(
                x/1000, x/1000, np.log10(data), vmin=vmin, vmax=vmax,
                linewidth=0, rasterized=True,
                cmap=plt.cm.get_cmap("RdBu_r", 8))

    return cf

In [ ]:
def plot_result_rel(depm, de3d, x, title, vmin=-15., vmax=-7., mode="log"):
    fig, axs = plt.subplots(figsize=(18, 10), nrows=2, ncols=3)

    if mode == "log":
        case = ""
    else:
        case = "|"

    # Plot Re(data)
    cf0 = plot_data_rel(axs[0, 0], r"(a) "+case+"Re(empymod)"+case,
                        depm.real, x, vmin, vmax, mode)
    plot_data_rel(axs[0, 1], r"(b) "+case+"Re(emg3d)"+case,
                  de3d.real, x, vmin, vmax, mode)
    cf2 = plot_data_rel(axs[0, 2], r"(c) Error real part",
                        np.abs((depm.real-de3d.real)/depm.real)*100, x,
                        vmin=-2, vmax=2, mode=True)

    # Plot Im(data)
    plot_data_rel(axs[1, 0], r"(d) "+case+"Im(empymod)"+case,
                  depm.imag, x, vmin, vmax, mode)
    plot_data_rel(axs[1, 1], r"(e) "+case+"Im(emg3d)"+case,
                  de3d.imag, x, vmin, vmax, mode)
    plot_data_rel(axs[1, 2], r"(f) Error imaginary part",
                  np.abs((depm.imag-de3d.imag)/depm.imag)*100,
                  x, vmin=-2, vmax=2, mode=True)

    # Colorbars
    fig.colorbar(cf0, ax=axs[0, :], label=r"$\log_{10}$ Amplitude (A/m)")
    cbar = fig.colorbar(cf2, ax=axs[1, :], label=r"Relative Error")
    cbar.set_ticks([-2, -1, 0, 1, 2])
    cbar.ax.set_yticklabels([r"$0.01\,\%$", r"$0.1\,\%$", r"$1\,\%$",
                             r"$10\,\%$", r"$100\,\%$"])

    # Axis label
    fig.text(0.4, 0.05, "Inline Offset (km)", fontsize=14)
    fig.text(0.08, 0.6, "Crossline Offset (km)", rotation=90, fontsize=14)

    # Title
    fig.suptitle(title, y=1, fontsize=20)
    plt.show()

In [ ]:
def plot_lineplot_ex(x, y, data, epm_fs, grid):
    xi = x.size//2
    yi = y.size//2

    fn = sint.interp1d(x, data[:, xi], bounds_error=False)
    # x1 = fn(grid.vectorNx)

    fn = sint.interp1d(y, data[yi, :], bounds_error=False)
    y1 = fn(grid.vectorNx)

    plt.figure(figsize=(15, 8))

    plt.plot(y/1e3, np.abs(epm_fs[yi, :]), 'C1', lw=3, label='Inline empymod')
    plt.plot(y/1e3, np.abs(data[yi, :]), 'k:', label='Inline emg3d')
    plt.plot(grid.vectorNx/1e3, np.abs(y1), 'k*', label='Grid points emg3d')

    plt.yscale('log')
    plt.title(r'Inline $H_x$', fontsize=20)
    plt.xlabel('Offset (km)', fontsize=14)
    plt.ylabel(r'|Amplitude (A/m)|', fontsize=14)
    plt.legend()
    plt.show()

Full-space model for a rotated magnetic loop
--------------------------------------------

empymod
```````



In [ ]:
# Survey parameters
x = (np.arange(1025))*5-2560
rx = np.repeat([x, ], np.size(x), axis=0)
ry = rx.transpose()

# Model parameters
resh = 1.              # Horizontal resistivity
src = [0, 0, -300, 10, 70]  # Source: [x, y, z, azimuth, dip]
zrec = -400.           # Receiver depth
freq = 0.77            # Frequency
strength = np.pi       # Source strength

# Input for empymod
model = {
    'src': src,
    'depth': [],
    'res': resh,
    'strength': strength,
    'freqtime': freq,
    'htarg': {'pts_per_dec': -1},
}

In [ ]:
rxx = rx.ravel()
ryy = ry.ravel()

# e-field
epm_fs_ex = empymod.loop(rec=[rxx, ryy, zrec, 0, 0], mrec=False, verb=3,
                         **model).reshape(np.shape(rx))
epm_fs_ey = empymod.loop(rec=[rxx, ryy, zrec, 90, 0], mrec=False, verb=1,
                         **model).reshape(np.shape(rx))
epm_fs_ez = empymod.loop(rec=[rxx, ryy, zrec, 0, 90], mrec=False, verb=1,
                         **model).reshape(np.shape(rx))

# h-field
epm_fs_hx = empymod.loop(rec=[rxx, ryy, zrec, 0, 0], verb=1,
                         **model).reshape(np.shape(rx))
epm_fs_hy = empymod.loop(rec=[rxx, ryy, zrec, 90, 0], verb=1,
                         **model).reshape(np.shape(rx))
epm_fs_hz = empymod.loop(rec=[rxx, ryy, zrec, 0, 90], verb=1,
                         **model).reshape(np.shape(rx))

emg3d
`````



In [ ]:
# Get computation domain as a function of frequency (resp., skin depth)
hx_min, xdomain = emg3d.meshes.get_domain(x0=src[0], freq=0.1, min_width=20)
hz_min, zdomain = emg3d.meshes.get_domain(x0=src[2], freq=0.1, min_width=20)

# Create stretched grid
nx = 2**7
hx = emg3d.meshes.get_stretched_h(hx_min, xdomain, nx, src[0])
hy = emg3d.meshes.get_stretched_h(hx_min, xdomain, nx, src[1])
hz = emg3d.meshes.get_stretched_h(hz_min, zdomain, nx, src[2])
pgrid = emg3d.TensorMesh([hx, hy, hz], x0=(xdomain[0], xdomain[0], zdomain[0]))
pgrid

Abuse the parameters to take advantage of the duality principle
---------------------------------------------------------------

See text at the top. We set here $\rho=1$ and $\mu_\mathrm{r} =
1/\rho$ to get:

\begin{align}:label: iweta2iwu

    \mathrm{i}\omega\mu_0(1-\mathrm{i}\omega\varepsilon) =
    \mathrm{i}\omega\mu_0+\omega^2\mu_0\varepsilon \approx
    \mathrm{i}\omega\mu\end{align}

(in the diffusive regime), and

\begin{align}:label: mu2sigma

    \mu_\mathrm{r} = 1/\rho = \sigma \, .\end{align}



In [ ]:
# Get the model        => Set property_x = 1 and mu_r = 1./resh
pmodel = emg3d.Model(pgrid, property_x=1., mu_r=1./resh, mapping='Resistivity')

In [ ]:
# Get the source field
sfield = emg3d.get_source_field(pgrid, src, freq, strength)

# Compute the electric field
hfield = emg3d.solve(pgrid, pmodel, sfield, verb=3)

Compare the magnetic field generated from the magnetic source
-------------------------------------------------------------



In [ ]:
e3d_fs_hx = emg3d.get_receiver(pgrid, hfield.fx, (rx, ry, zrec))
plot_result_rel(epm_fs_hx, e3d_fs_hx, x, r'Diffusive Fullspace $H_x$',
                vmin=-15, vmax=-8, mode='abs')

In [ ]:
e3d_fs_hy = emg3d.get_receiver(pgrid, hfield.fy, (rx, ry, zrec))
plot_result_rel(epm_fs_hy, e3d_fs_hy, x, r'Diffusive Fullspace $H_y$',
                vmin=-15, vmax=-8, mode='abs')

In [ ]:
e3d_fs_hz = emg3d.get_receiver(pgrid, hfield.fz, (rx, ry, zrec))
plot_result_rel(epm_fs_hz, e3d_fs_hz, x, r'Diffusive Fullspace $H_z$',
                vmin=-14, vmax=-7, mode='abs')

In [ ]:
plot_lineplot_ex(x, x, e3d_fs_hx.real, epm_fs_hx.real, pgrid)

Compare the electric field generated from the magnetic source
-------------------------------------------------------------

``get_h_field`` gets the $H$-field from the $E$-field with
Faraday's law,

\begin{align}:label: faraday2

    \nabla \times \mathbf{E} = \rm{i}\omega \mathbf{B} =
    \rm{i}\omega\mu\mathbf{H}\, .\end{align}

Using the substitutions introduced in the beginning, and using the same
function but to get the $E$-field from the $H$-field, we have to
multiply the result by

\begin{align}:label: iwu

    -\rm{i}\omega\mu\, .\end{align}

Compute electric field $E$ from the magnetic field
````````````````````````````````````````````````````````



In [ ]:
efield = emg3d.get_h_field(
        pgrid, pmodel, hfield)*2j*np.pi*freq*4e-7*np.pi

Plot
````



In [ ]:
e3d_fs_ex = emg3d.get_receiver(pgrid, efield.fx, (rx, ry, zrec))
plot_result_rel(epm_fs_ex, e3d_fs_ex, x, r'Diffusive Fullspace $E_x$',
                vmin=-17, vmax=-10, mode='abs')

In [ ]:
e3d_fs_ey = emg3d.get_receiver(pgrid, efield.fy, (rx, ry, zrec))
plot_result_rel(epm_fs_ey, e3d_fs_ey, x, r'Diffusive Fullspace $E_y$',
                vmin=-17, vmax=-10, mode='abs')

In [ ]:
e3d_fs_ez = emg3d.get_receiver(pgrid, efield.fz, (rx, ry, zrec))
plot_result_rel(epm_fs_ez, e3d_fs_ez, x, r'Diffusive Fullspace $E_z$',
                vmin=-17, vmax=-10, mode='abs')

In [ ]:
emg3d.Report()